### *Nature Climate Change Commentary on the IPCC SR1.5 scenario resource*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# An overview of the *IAMC 1.5°C scenario ensemble*

This notebook generates the figure in the commentary published in *Nature Climate Change*
on the scenario resource compiled for the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario ensemble can be accessed at https://data.ene.iiasa.ac.at/iamc-1.5c-explorer.

#### License and recommended citation

This notebook is licensed under the APACHE 2.0 License. Please read the [NOTICE](NOTICE) for more information.

When using this notebook, please cite as:

>    Daniel Huppmann et al.,
>    Scenario analysis notebooks for the IPCC Special Report on of Global Warming of 1.5°C. 2018  
>    doi: [10.22022/SR15/08-2018.15428](https://doi.org/10.22022/SR15/08-2018.15428) |
>    url: [github.com/iiasa/ipcc_sr15_scenario_analysis](https://github.com/iiasa/ipcc_sr15_scenario_analysis)

You can download the recommended citations as [Endnote (ris)](../bibliography/iamc_15c.ris)
or [BibTex (bib)](../bibliography/iamc_15c.bib) format.

## Load required packages, import data and metadata

The metadata file should have been generated from the notebook ``sr1p5_categories_indicators`` included in this repository.  
If the snapshot file has been updated, make sure that you have an up-to-date metadata file.

The last cell of this section loads and assigns a number of auxiliary lists and the `run_control` specifications as defined in the categories-indicators notebook. The imported dictionary `specs` should be empty after popping all relevant data.

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
%matplotlib inline
import pyam

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_world_public_release_v0.csv')
sr1p5.filter(region='World', inplace=True)

In [ ]:
sr1p5.load_meta('../analysis/sr1p5_metadata_indicators.xlsx')

In [ ]:
with open("../analysis/sr1p5_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
plotting_args = specs.pop('plotting_args')
marker= specs.pop('marker')

specs

## Filter for valid scenarios

Only include relevant scenarios in the analysis.

In [ ]:
full_century = range(2000, 2101, 10)
horizon = range(2010, 2101, 10)

In [ ]:
df = sr1p5.filter(category=cats, year=full_century, exclude=False)

## Plots of drivers

In [ ]:
pop = df.filter(variable='Population')
pop.convert_unit('million', 'billion', factor=1/1000, inplace=True)

gdp = df.filter(variable='GDP|PPP')
gdp.convert_unit('billion US$2010/yr', 'trillion US$2010/yr', factor=1/1000), inplace=True)

primary = df.filter(variable='Primary Energy')

forest =  df.filter(variable='Land Cover|Forest')
forest.convert_unit('million ha', 'million km2', factor=1/100, inplace=True)

In [ ]:
config = [
    (pop, 'Population'),
    (gdp, 'Economic output'),
    (primary, 'Primary energy'),
    (forest, 'Forest cover')
]

drivers, _ax = plt.subplots(1, 4, figsize=(11, 2))

for i, (_df, title) in enumerate(config):
    
    _df = _df.filter(year=horizon)
    data = _df.timeseries()
    data = data[data[2020] > 1]
    _min = data.quantile(q=0)
    _max = data.quantile(q=1)
    _ax[i].stackplot(horizon, _min, _max - _min, colors=['', 'lightgrey'])
    
    _ax[i] = _df.line_plot(ax=_ax[i], color='lightskyblue', linewidth=0.5, legend=False)
    _ax[i].set_title(title)
    _ax[i].set_xlabel('')
    _ax[i].set_ylabel(_df.data.unit.unique()[0])
    _ax[i].set_ylim(0.9 * min(_min), 1.1 * max(_max))

drivers.tight_layout()

In [ ]:
drivers.savefig('ncc_drivers.eps')

## Carbon dioxide emission

In [ ]:
co2 = df.filter(year=horizon, variable='Emissions|CO2')
co2.convert_unit('Mt CO2/yr', 'Gt CO2/yr', inplace=True)

In [ ]:
pyam.run_control()

In [ ]:
emissions, _ax = plt.subplots(1, 2, sharey=True, figsize=(6, 2))

classes = [
    ('1.5C pathways', ['Below 1.5C', '1.5C low OS', '1.5C high OS'], 'xkcd:bluish'),
    ('2C pathways', ['Lower 2C', 'Higher 2C'], 'xkcd:orange'),
    ('Exceeding 2C', ['Above 2C'], 'lightgrey'),
]

## settings for axes 2

years = [2030, 2050, 2100]
ymax = 70
mincount = 7

_classes = len(classes) - 1
w = 0.6 / _classes

_ax[1].hlines(0, xmin=-0.5, xmax=(len(years) - 0.5), linewidths=0.6)

##

for (name, _cats, c) in reversed(classes):
    _co2 = co2.filter(category=_cats).timeseries()
    if not _co2.empty:
        _min = _co2.quantile(q=0.25)
        _max = _co2.quantile(q=0.75)
        _med = _co2.apply(lambda x: np.median(x))
        _ax[0].stackplot(horizon, _min, _max - _min, colors=['', c])
        _ax[0].plot(horizon, _min, color=c, linewidth=0.8)
        _ax[0].plot(horizon, _max, color=c, linewidth=0.8)
        _ax[0].plot(horizon, _med, color=c, linewidth=2)

for i, (name, _cats, c) in enumerate(classes):
    _co2 = co2.filter(category=_cats).timeseries()
    if not _co2.empty:
        for j, y in enumerate(years):
            lst = _co2[y][~np.isnan(_co2[y])]
            pos = (0.75 / _classes * (i - _classes / 2) + j)

            outliers = len(lst[lst > ymax])
#            if outliers > 0:
#                _ax[1].text(pos - 0.01 * len(years), ymax, outliers)

            if len(lst) >= mincount:
                p = _ax[1].boxplot(lst, positions=[pos], widths=w * .90,
                                whis='range',
                                patch_artist=True)
                plt.setp(p['boxes'], color=c)
                plt.setp(p['medians'], color='black')
            else:
                _ax[1].scatter(x=[pos] * len(lst), y=lst, zorder=5,
                            c=c, edgecolors='black', marker='o',
                            s=30, label=None)
                _ax[1].plot([pos, pos], [max(lst), min(lst)], zorder=4,
                         color='black', linewidth=1, linestyle='-',
                         marker='_', markersize=8, markeredgewidth=1,
                         markeredgecolor='black')

        _ax[1].plot([], c=c, label='{} [{}]'.format(name, len(_co2)))

_ax[0].hlines(0, xmin=horizon.start, xmax=horizon.stop - 1, linewidths=0.6)
#_ax[0].set_title('Trajectories by category')
_ax[0].set_ylabel('Gt CO2/yr')
_ax[0].set_ylim(-21, ymax)

_ax[1].set_position([0.49, 0.124, 0.2, 0.753])
#_ax[1].set_title('Ranges')
_ax[1].set_xlim(-0.6, (len(years) - 0.4))
plt.xticks(range(0, len(years)), years)

_ax[1].legend()
#emissions.tight_layout()

In [ ]:
emissions.savefig('ncc_carbon_emissions.eps')

## Statistical overview of bioenergy and other indicators

In [ ]:
ccs = df.filter(variable='Carbon Sequestration|CCS')
ccs.convert_unit('Mt CO2/yr', 'Gt CO2/yr', inplace=True)
beccs = df.filter(variable='Carbon Sequestration|CCS|Biomass')
beccs.convert_unit('Mt CO2/yr', 'Gt CO2/yr', inplace=True)

In [ ]:
cum_ccs = ccs.timeseries().apply(pyam.cumulative, raw=False, axis=1, first_year=2020, last_year=2100)
cum_beccs = beccs.timeseries().apply(pyam.cumulative, raw=False, axis=1, first_year=2020, last_year=2100)

In [ ]:
ccs_fig, _ax = plt.subplots(2, 2, sharey=True, figsize=(4, 2))

for i, (_cats, superclass, color) in enumerate([(['Below 1.5C', '1.5C low OS', '1.5C high OS'], '1.5', 'palegreen'),
                                         (['Lower 2C', 'Higher 2C'], '2.0', 'palevioletred')]):

    for j, (data, group, count) in enumerate([(cum_ccs, 'CCS', 20),
                                      (cum_beccs, 'BECCS', 12)]):
        groupby = pyam.filter_by_meta(data.reset_index(),df, join_meta=True, category=_cats).groupby('category')

        data = []
        color = []
        labels = []

        for name in _cats:
            x = groupby.get_group(name)
            if not x.empty:
                data.append(x[0][~np.isnan(x[0])])
                color.append(rc['color']['category'][name])
                labels.append([name])

            _ax[i][j].hist(data, count, color=color, label=labels, stacked=True)

        if i == 0:
            _ax[i][j].set_title(group)
            _ax[i][j].set_xticks([])
        else:
            _ax[1][j].set_xlabel('GtCO2')
    _ax[i][0].set_ylabel('Number of \n {}°C scenarios'.format(superclass))

_ax[0][1].set_position([0.52, 0.538, 0.2, 0.342])
_ax[1][1].set_position([0.52, 0.124, 0.2, 0.342])

## Energy system transformation in two illustrative pathways

In [ ]:
marker = ['LED', 'S5']

In [ ]:
ene = sr1p5.filter(marker=marker)

In [ ]:
variable_mapping = [
    ('Fossil without CCS', 'Primary Energy|Fossil|w/o CCS', 'black', ''),
    ('Fossil with CCS', 'Primary Energy|Fossil|w/ CCS', 'grey', '//'),
    ('Biomass without CCS',
         ['Primary Energy|Biomass|Modern|w/o CCS',
          'Primary Energy|Biomass|Traditional'], 'forestgreen', ''),
    ('Biomass with CCS', 'Primary Energy|Biomass|Modern|w/ CCS', 'limegreen', '//'),
    ('Nuclear', 'Primary Energy|Nuclear', 'firebrick', ''),
    ('Other renewables',
         ['Primary Energy|Ocean',
          'Primary Energy|Geothermal',
          'Primary Energy|Hydro'], 'darkorange', ''),
    ('Wind', 'Primary Energy|Wind', 'lightskyblue', ''),
    ('Solar', 'Primary Energy|Solar', 'gold', ''),
]

In [ ]:
variables = []
mapping = {}

for (name, variable, color, hatch) in variable_mapping:
    variables.append(name)
    if isinstance(variable, list):
        for v in variable:
            mapping.update({v: name})
    else:
        mapping.update({variable: name})
    rc.update({'color': {'variable': {name: color}}})
    rc.update({'hatch': {'variable': {name: hatch}}})

ene.rename({'variable': mapping}, inplace=True)

In [ ]:
_ene = ene.filter(variable=variables, year=range(2010, 2101, 10))

In [ ]:
ene_fig, _ax = plt.subplots(1, len(marker), sharey=True, figsize=(15, 3))

for i, m in enumerate(marker):
    _df = _ene.filter(marker=m)
    _df.data.variable = _df.data.variable.astype('category')
    _df.data.variable.cat.set_categories(variables, inplace=True)
    _df.data.sort_values('variable', inplace=True)
    _df.stack_plot(ax=_ax[i], legend=False)
    _ax[i].set_title('')
    _ax[i].set_xlabel('')
    if i > 0:
        _ax[i].set_ylabel('')
_ax[0].legend()

In [ ]:
ene_fig.savefig('ncc_energy_transitions.eps')